In [ ]:
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

In [16]:
import os
from langchain_groq import ChatGroq

TEMPERATURE=0.5
LLM_MODEL_NAME="mixtral-8x7b-32768"
EMBED_MODEL_NAME="sentence-transformers/all-MiniLM-L6-v2"
MILVUS_URI="./milvus_example.db"

In [17]:
llm = ChatGroq(temperature=TEMPERATURE, model_name=LLM_MODEL_NAME, api_key=os.getenv("GROQ_API_KEY"))
llm.invoke("hi")

AIMessage(content="Hello! How can I help you today? If you have any questions or need assistance withUserGuiding, just let me know. I'm here to make sure you have the best possible experience. 😊", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 8, 'total_tokens': 52, 'completion_time': 0.066706525, 'prompt_time': 0.001768253, 'queue_time': 0.06504906, 'total_time': 0.068474778}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-1735331d-6daf-4919-985f-b25c1e6e2b0a-0', usage_metadata={'input_tokens': 8, 'output_tokens': 44, 'total_tokens': 52})

In [ ]:
llm.invoke("hi")

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [ ]:
model.invoke("Hello, world!")

In [18]:
from langchain.schema import HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [20]:
AIMessage(content="Hi i am an AI.").content

'Hi i am an AI.'